In [ ]:
%%HTML <style>pre { white-space: pre !important; }</style>

In [ ]:
import org.apache.spark.sql.{SparkSession, DataFrame}

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
val clientsDf = spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","true")
        .csv("../../resources/data/csv/clients.csv")
val contractsDf = spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","true")
        .csv("../../resources/data/csv/contracts.csv")

clientsDf.show(2)
contractsDf.show(2)

#### Transformaciones

In [ ]:
/*
Joins

inner -> Mantiene información de ambas tablas (columnas) para los registros (filas) coincidentes
outer -> Mantiene información de ambas tablas (columnas y filas) para los registros coincidentes y no-coincidentes
left -> Mantiene columnas de ambas tablas y filas únicamente de la tabla izquierda, elimina filas no coincidentes de la tabla derecha
right -> Mantiene columnas de ambas tablas y filas únicamente de la tabla derecha, elimina filas no coincidentes de la tabla izquierda
left_semi -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros coincidentes
left_anti -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros no-coincidentes

cross
*/

In [ ]:
import org.apache.spark.sql.{functions => f}

val clientsTmpDf = clientsDf.filter((f.col("edad") >= 40) && (f.col("edad") <= 50))
val contractsTmpDf = contractsDf.filter(f.col("activo") === false)
    .withColumnRenamed("cod_titular", "cod_client")

clientsTmpDf.show()
contractsTmpDf.show()

val typeJoin = "full"   // inner, outer, left, right, left_semi, left_anti

val joinDf = clientsTmpDf.join(contractsTmpDf, Seq("cod_client"), typeJoin)
joinDf.show()

In [ ]:
clientsTmpDf.crossJoin(contractsTmpDf).show() // WARNING

In [ ]:
// UDF - User Defined Function - WARNING
import org.apache.spark.sql.{types => t}
import scala.util.Try

val upperCaseFunction: Option[String] => String = value => value.getOrElse("").toUpperCase
val upperCaseFunction2: String => Option[String] = value => Try(value.toString.toUpperCase).toOption



val len_concat: (Option[String], Option[String]) => Int = (item_1, item_2) => {
    val EmptyString = ""
    val str1 = item_1.getOrElse(EmptyString)
    val str2 = item_2.getOrElse(EmptyString)
    
    str1.concat(str2).size
}
val len_concat2: (String, String) => Option[Int] = (item_1, item_2) => {
    Try(item_1.concat(item_2).size).toOption
}



def sumValues: (Option[Int], Option[Int]) => Long = (value_1, value_2) => {
    value_1.getOrElse(0) + value_2.getOrElse(0)
}
def sumValues2: (Int, Int) => Long = (value_1, value_2) => {
    value_1 + value_2
}



val upperUdf = f.udf[String, Option[String]](upperCaseFunction)

val lenConcatUdf = f.udf(len_concat2)

val sumValuesUdf = f.udf[Long, Option[Int], Option[Int]](sumValues)

joinDf.select(
    joinDf.columns.map(f.col) :+
    upperUdf(f.col("nombre")).alias("nombre_mayus") :+
    sumValuesUdf(f.col("edad"), f.col("cod_producto")).alias("sum_values") :+
    lenConcatUdf(f.col("nombre"), f.col("provincia")).alias("len_concat") :_*
).show()

In [ ]:
joinDf.createOrReplaceTempView("view")

spark.udf.register[Option[Int], String, String]("MY_UDF_NAME", len_concat2)

spark.sql("""
    SELECT MY_UDF_NAME(nombre, provincia), nombre, provincia FROM view
""").show()